# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

***
*WORK IN PROGRESS*
***

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hopspitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hopspitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-23-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-23-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-06-24 04:33:28,34.223334,-82.461707,86,0,0,86,"Abbeville, South Carolina, US",350.633995,0.000000
1,22001.0,Acadia,Louisiana,US,2020-06-24 04:33:28,30.295065,-92.414197,705,34,0,671,"Acadia, Louisiana, US",1136.272061,4.822695
2,51001.0,Accomack,Virginia,US,2020-06-24 04:33:28,37.767072,-75.632346,1033,14,0,1019,"Accomack, Virginia, US",3196.558980,1.355276
3,16001.0,Ada,Idaho,US,2020-06-24 04:33:28,43.452658,-116.241552,1307,22,0,1285,"Ada, Idaho, US",271.394369,1.683244
4,19001.0,Adair,Iowa,US,2020-06-24 04:33:28,41.330756,-94.471059,12,0,0,12,"Adair, Iowa, US",167.785235,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until day is over.**

In [6]:
jh_cases_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_cases_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,24766,25527,26310,26874,27532,27878,28424,28833,29157,29481
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1521,1590,1672,1722,1788,1838,1891,1962,1995,2047
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,10919,11031,11147,11268,11385,11504,11631,11771,11920,12076


In [7]:
jh_deaths_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_deaths_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,471,478,491,504,546,548,569,581,598,618
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,36,36,37,38,39,42,43,44,44,45
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,767,777,788,799,811,825,837,845,852,861


In [8]:
jh_recovered_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_recovered_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,4725,5164,5508,6158,7660,7962,8292,8764,8841,9260
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1044,1055,1064,1077,1086,1114,1126,1134,1159,1195
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,7606,7735,7842,7943,8078,8196,8324,8422,8559,8674


In [9]:
jh_cases_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_cases_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,183,185,186,188,192,200,222,222,222,225
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,30,30,30,30,30,30,30,30,30,30


In [10]:
jh_deaths_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_deaths_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [11]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
1313,1071.0,Jackson,Alabama,US,2020-06-24 04:33:28,34.781442,-85.997505,159,3,0,156,"Jackson, Alabama, US",307.984349,1.886792
2177,1107.0,Pickens,Alabama,US,2020-06-24 04:33:28,33.280949,-88.088181,171,6,0,165,"Pickens, Alabama, US",858.003011,3.508772
685,1045.0,Dale,Alabama,US,2020-06-24 04:33:28,31.430371,-85.610957,206,0,0,206,"Dale, Alabama, US",418.937607,0.000000


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [12]:
jh_countries_live = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [13]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,2347022,121228,647548
Brazil,1145906,52645,627963
Russia,598878,8349,355847


In [14]:
jh_countries_live['Active'] = jh_countries_live['Confirmed'] - jh_countries_live['Deaths'] - jh_countries_live['Recovered']

In [15]:
jh_countries_live['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_countries_live['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_countries_live['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [16]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,2347022,121228,647548,1578246,2020-06-24 04:33:28,38.258814,-91.492718
Brazil,1145906,52645,627963,465298,2020-06-24 04:33:28,-12.669522,-48.480493
Russia,598878,8349,355847,234682,2020-06-24 04:33:28,54.546312,62.120860
India,456183,14476,258685,183022,2020-06-24 04:33:28,23.356511,82.081064
United Kingdom,307682,43011,1330,263341,2020-06-24 04:33:28,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [17]:
jh_states_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [18]:
jh_states_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,389085,31232,0
California,191477,5626,0
New Jersey,169734,13025,0


In [19]:
jh_states_live['Active'] = jh_states_live['Confirmed'] - jh_states_live['Deaths'] - jh_states_live['Recovered']

In [20]:
jh_states_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_states_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_states_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [21]:
jh_states_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,389085,31232,0,357853,2020-06-24 04:33:28,42.671593,-75.579694
California,191477,5626,0,185851,2020-06-24 04:33:28,37.778248,-120.728666
New Jersey,169734,13025,0,156709,2020-06-24 04:33:28,40.286441,-74.622450
Illinois,137825,6707,0,131118,2020-06-24 04:33:28,39.843618,-89.178330
Texas,122932,2236,0,120696,2020-06-24 04:33:28,31.668555,-98.559966


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [22]:
jh_counties_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

Confirmed  Deaths  Recovered
Province_State Admin2                                     
New York       New York City     213056   22343          0
California     Los Angeles        88512    3171          0
Illinois       Cook               87784    4439          0
New York       Nassau             41544    2179          0
               Suffolk            41056    1970          0

In [24]:
jh_counties_live['Active'] = jh_counties_live['Confirmed'] - jh_counties_live['Deaths'] - jh_counties_live['Recovered']


In [25]:
jh_counties_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_counties_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_counties_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_counties_live['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [26]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

Confirmed  Deaths  Recovered  Active  \
Province_State Admin2                                                
New York       New York City     213056   22343          0  190713   
California     Los Angeles        88512    3171          0   85341   
Illinois       Cook               87784    4439          0   83345   
New York       Nassau             41544    2179          0   39365   
               Suffolk            41056    1970          0   39086   

                                      Last_Update        Lat        Long  \
Province_State Admin2                                                      
New York       New York City  2020-06-24 04:33:28  40.767273  -73.971526   
California     Los Angeles    2020-06-24 04:33:28  34.308284 -118.228241   
Illinois       Cook           2020-06-24 04:33:28  41.841448  -87.816588   
New York       Nassau         2020-06-24 04:33:28  40.740665  -73.589419   
               Suffolk        2020-06-24 04:33:28  40.883201  -72.801217   

                                 FIPS  
Province_State Admin2                  
New York       New York City  36061.0  
California     Los Angeles     6037.0  
Illinois       Cook           17031.0  
New York       Nassau         36059.0  
               Suffolk        36103.0

##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [27]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_cases_global_hist.iloc[:, 4:].columns

In [28]:
jh_cases_countries_hist_ts = pd.pivot_table(jh_cases_global_hist, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.transpose()
jh_cases_countries_hist_ts.index = pd.to_datetime(jh_cases_countries_hist_ts.index)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.sort_index()

In [29]:
jh_cases_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-21,28833,1962,11771,855,183,26,42785,20268,7474,17341,...,305803,876,6315,3917,349,833,9,941,1430,489
2020-06-22,29157,1995,11920,855,186,26,44931,20588,7492,17380,...,306761,882,6461,4048,349,1001,10,967,1430,512
2020-06-23,29481,2047,12076,855,189,26,47203,21006,7521,17408,...,307682,885,6662,4187,349,1169,10,992,1477,525


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [30]:
jh_deaths_countries_hist_ts = pd.pivot_table(jh_deaths_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.transpose()
jh_deaths_countries_hist_ts.index = pd.to_datetime(jh_deaths_countries_hist_ts.index)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.sort_index()

In [31]:
jh_deaths_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-21,581,44,845,52,9,3,1011,350,102,690,...,42717,25,19,33,0,3,1,256,11,6
2020-06-22,598,44,852,52,10,3,1043,360,102,690,...,42731,25,19,35,0,3,1,257,11,6
2020-06-23,618,45,861,52,10,3,1078,372,103,693,...,43011,25,19,35,0,3,1,261,18,6


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [32]:
jh_recovered_countries_hist_ts = pd.pivot_table(jh_recovered_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.transpose()
jh_recovered_countries_hist_ts.index = pd.to_datetime(jh_recovered_countries_hist_ts.index)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.sort_index()

In [33]:
jh_recovered_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-21,8764,1134,8422,792,77,22,12728,9002,6903,16197,...,1319,814,4377,835,327,439,8,347,1194,64
2020-06-22,8841,1159,8559,796,77,22,13153,9131,6915,16241,...,1322,815,4450,1327,328,442,8,350,1194,64
2020-06-23,9260,1195,8674,797,77,22,13576,10144,6924,16261,...,1330,815,4560,1327,329,442,8,356,1213,64


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [34]:
jh_cases_counties_hist.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,882,896,926,956,994,1031,1054,1075,1097,1140
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,140,142,145,149,161,165,168,168,168,171
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,209,221,222,225,234,237,237,237,237,243
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,125,126,132,133,136,138,144,147,148,147


In [35]:
date_columns = jh_cases_counties_hist.iloc[:,11:].columns

In [36]:
jh_cases_states_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.transpose()
jh_cases_states_hist_ts.index = pd.to_datetime(jh_cases_states_hist_ts.index)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.sort_index()

In [37]:
jh_cases_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-21,30021,752,0,52591,15561,178546,30524,45755,10775,49,...,35102,112944,17462,1159,76,57994,28680,2533,24819,1197
2020-06-22,30454,758,0,54599,16083,184765,30689,45782,10820,49,...,35553,117790,17906,1163,76,58465,28870,2552,25068,1230
2020-06-23,31097,760,0,58378,16678,191477,30877,45899,10847,49,...,36303,122932,18300,1164,76,58994,29386,2593,25331,1254


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [38]:
jh_cases_counties_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.transpose()
jh_cases_counties_hist_ts.index = pd.to_datetime(jh_cases_counties_hist_ts.index)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.sort_index()

In [39]:
jh_cases_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-21         428     415     271  124    146     324    574     205   
2020-06-22         436     422     276  126    150     325    576     207   
2020-06-23         447     435     279  132    156     325    579     208   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-21          507       56  ...      11      1       19        4   
2020-06-22          514       56  ...      12      2       19        5   
2020-06-23          520       56  ...      16      2       19        5   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-21             53   110   138          0       39      1  
2020-06-22             56   111   148          0       39      1  
2020-06-23             58   113   152          0       39      1  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [40]:
jh_deaths_states_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.transpose()
jh_deaths_states_hist_ts.index = pd.to_datetime(jh_deaths_states_hist_ts.index)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.sort_index()

In [41]:
jh_deaths_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-21,839,12,0,1349,225,5515,1647,4260,435,0,...,526,2191,158,56,6,1611,1270,89,744,20
2020-06-22,841,12,0,1351,227,5566,1651,4263,435,0,...,531,2207,158,56,6,1620,1276,89,745,20
2020-06-23,864,12,0,1396,237,5626,1665,4277,504,0,...,542,2236,163,56,6,1645,1284,92,750,20


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [42]:
jh_deaths_counties_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.transpose()
jh_deaths_counties_hist_ts.index = pd.to_datetime(jh_deaths_counties_hist_ts.index)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.sort_index()

In [43]:
jh_deaths_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-21           8       9       1    1      1       9     26       4   
2020-06-22           8       9       1    1      1       9     26       4   
2020-06-23           8       9       1    1      1       9     27       5   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-21           27        6  ...       0      0        0        0   
2020-06-22           27        6  ...       0      0        0        0   
2020-06-23           27        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-21              0     0     0         19        0      0  
2020-06-22              0     0     0         19        0      0  
2020-06-23              0     0     0         19        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [44]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_countries_live.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [45]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_states_live.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [46]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_counties_live.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [47]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_cases_countries_hist_ts.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [48]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_deaths_countries_hist_ts.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_recovered_countries_hist_ts.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_cases_states_hist_ts.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_cases_counties_hist_ts.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_deaths_states_hist_ts.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_deaths_counties_hist_ts.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [54]:
jh_cases_countries_hist_pct = jh_cases_countries_hist_ts.pct_change()

In [55]:
jh_cases_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-21,0.014389,0.037546,0.012037,0.0,0.039773,0.0,0.038370,0.028415,0.001742,0.001039,...,0.004015,0.019790,0.026329,0.033782,0.0,0.062500,0.000000,0.020607,0.000000,0.020877
2020-06-22,0.011237,0.016820,0.012658,0.0,0.016393,0.0,0.050158,0.015788,0.002408,0.002249,...,0.003133,0.006849,0.023120,0.033444,0.0,0.201681,0.111111,0.027630,0.000000,0.047035
2020-06-23,0.011112,0.026065,0.013087,0.0,0.016129,0.0,0.050566,0.020303,0.003871,0.001611,...,0.003002,0.003401,0.031110,0.034338,0.0,0.167832,0.000000,0.025853,0.032867,0.025391


In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_cases_countries_hist_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [57]:
jh_deaths_countries_hist_pct = jh_deaths_countries_hist_ts.pct_change()

In [58]:
jh_deaths_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-21,0.021090,0.023256,0.009558,0.0,0.000000,0.0,0.019153,0.054217,0.000000,0.002907,...,0.001008,0.0,0.0,0.000000,NaN,0.0,0.0,0.007874,0.000000,0.5
2020-06-22,0.029260,0.000000,0.008284,0.0,0.111111,0.0,0.031652,0.028571,0.000000,0.000000,...,0.000328,0.0,0.0,0.060606,NaN,0.0,0.0,0.003906,0.000000,0.0
2020-06-23,0.033445,0.022727,0.010563,0.0,0.000000,0.0,0.033557,0.033333,0.009804,0.004348,...,0.006553,0.0,0.0,0.000000,NaN,0.0,0.0,0.015564,0.636364,0.0


In [59]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_deaths_countries_hist_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [60]:
jh_recovered_countries_hist_pct = jh_recovered_countries_hist_ts.pct_change()

In [61]:
jh_recovered_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-21,0.056922,0.007105,0.011773,0.000000,0.166667,0.0,0.042766,0.016716,0.001015,0.001360,...,0.000000,-0.001227,0.020280,0.000000,0.000000,0.004577,0.0,0.057927,0.000000,0.015873
2020-06-22,0.008786,0.022046,0.016267,0.005051,0.000000,0.0,0.033391,0.014330,0.001738,0.002717,...,0.002274,0.001229,0.016678,0.589222,0.003058,0.006834,0.0,0.008646,0.000000,0.000000
2020-06-23,0.047393,0.031061,0.013436,0.001256,0.000000,0.0,0.032160,0.110941,0.001302,0.001231,...,0.006051,0.000000,0.024719,0.000000,0.003049,0.000000,0.0,0.017143,0.015913,0.000000


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_recovered_countries_hist_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [63]:
jh_cases_states_hist_pct = jh_cases_states_hist_ts.pct_change()

In [64]:
jh_cases_states_hist_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-21,0.015973,0.014845,NaN,0.049155,0.027671,0.019023,0.006297,0.000875,0.008801,0.0,...,0.019044,0.030690,0.023084,0.010462,0.041096,0.009592,0.016120,0.013200,0.011410,0.015267
2020-06-22,0.014423,0.007979,NaN,0.038181,0.033545,0.034831,0.005406,0.000590,0.004176,0.0,...,0.012848,0.042906,0.025427,0.003451,0.000000,0.008122,0.006625,0.007501,0.010033,0.027569
2020-06-23,0.021114,0.002639,NaN,0.069214,0.036996,0.036327,0.006126,0.002556,0.002495,0.0,...,0.021095,0.043654,0.022004,0.000860,0.000000,0.009048,0.017873,0.016066,0.010491,0.019512


In [65]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_cases_states_hist_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [66]:
jh_cases_counties_hist_pct = jh_cases_counties_hist_ts.pct_change()

In [67]:
jh_cases_counties_hist_pct.tail(3)

Province_State   Alabama                                                   \
Admin2           Autauga   Baldwin  Barbour      Bibb    Blount   Bullock   
2020-06-21      0.007059  0.024691  0.00000  0.008130  0.042857  0.000000   
2020-06-22      0.018692  0.016867  0.01845  0.016129  0.027397  0.003086   
2020-06-23      0.025229  0.030806  0.01087  0.047619  0.040000  0.000000   

Province_State                                         ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers Cherokee  ...      Park Platte   
2020-06-21      0.007018  0.009852  0.009960      0.0  ...  0.000000    0.0   
2020-06-22      0.003484  0.009756  0.013807      0.0  ...  0.090909    1.0   
2020-06-23      0.005208  0.004831  0.011673      0.0  ...  0.333333    0.0   

Province_State                                                              \
Admin2         Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-06-21          0.0     0.00   0.081633  0.009174  0.061538        NaN   
2020-06-22          0.0     0.25   0.056604  0.009091  0.072464        NaN   
2020-06-23          0.0     0.00   0.035714  0.018018  0.027027        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-06-21          0.0    0.0  
2020-06-22          0.0    0.0  
2020-06-23          0.0    0.0  

[3 rows x 3254 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_cases_counties_hist_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_deaths_states_hist_pct = jh_deaths_states_hist_ts.pct_change()

In [70]:
jh_deaths_states_hist_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-21,0.001193,0.0,NaN,0.002229,0.004464,0.003822,0.000000,0.002117,0.002304,NaN,...,0.003817,0.003665,0.019355,0.0,0.0,0.002489,0.003953,0.011364,0.000000,0.0
2020-06-22,0.002384,0.0,NaN,0.001483,0.008889,0.009248,0.002429,0.000704,0.000000,NaN,...,0.009506,0.007303,0.000000,0.0,0.0,0.005587,0.004724,0.000000,0.001344,0.0
2020-06-23,0.027348,0.0,NaN,0.033309,0.044053,0.010780,0.008480,0.003284,0.158621,NaN,...,0.020716,0.013140,0.031646,0.0,0.0,0.015432,0.006270,0.033708,0.006711,0.0


In [71]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_deaths_states_hist_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [72]:
jh_deaths_counties_hist_pct = jh_deaths_counties_hist_ts.pct_change()

In [73]:
jh_deaths_counties_hist_pct.tail(3)

Province_State Alabama                                                        \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock    Butler Calhoun   
2020-06-21         0.0     0.0     0.0  0.0    0.0     0.0  0.000000    0.00   
2020-06-22         0.0     0.0     0.0  0.0    0.0     0.0  0.000000    0.00   
2020-06-23         0.0     0.0     0.0  0.0    0.0     0.0  0.038462    0.25   

Province_State                     ... Wyoming                           \
Admin2         Chambers  Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-21          0.0  0.000000  ...     NaN    NaN      NaN      NaN   
2020-06-22          0.0  0.000000  ...     NaN    NaN      NaN      NaN   
2020-06-23          0.0  0.166667  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-21            NaN   NaN   NaN        0.0      NaN    NaN  
2020-06-22            NaN   NaN   NaN        0.0      NaN    NaN  
2020-06-23            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3254 columns]

In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_deaths_counties_hist_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [75]:
jh_cases_countries_hist_pct_ewma = jh_cases_countries_hist_pct.ewm(span=3).mean()

In [76]:
jh_cases_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-21,0.016795,0.034237,0.011397,0.000093,0.037252,9.536743e-09,0.042411,0.028648,0.002911,0.001792,...,0.004135,0.012192,0.028901,0.039528,0.003967,0.095325,9.313226e-10,0.016059,0.002128,0.025904
2020-06-22,0.014016,0.025528,0.012028,0.000046,0.026823,4.768372e-09,0.046284,0.022218,0.002660,0.002020,...,0.003634,0.009521,0.026010,0.036486,0.001984,0.148503,5.555556e-02,0.021844,0.001064,0.036470
2020-06-23,0.012564,0.025797,0.012558,0.000023,0.021476,2.384186e-09,0.048425,0.021261,0.003265,0.001816,...,0.003318,0.006461,0.028560,0.035412,0.000992,0.158168,2.777778e-02,0.023849,0.016965,0.030930


In [77]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_cases_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [78]:
jh_deaths_countries_hist_pct_ewma = jh_deaths_countries_hist_pct.ewm(span=3).mean()

In [79]:
jh_deaths_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-21,0.027451,0.030162,0.012355,0.000306,0.045900,3.414602e-21,0.021483,0.044885,-4.544152e-09,0.001909,...,0.002218,0.011096,0.000018,0.036255,NaN,4.656613e-10,0.0,0.016852,0.000417,0.2500
2020-06-22,0.028355,0.015081,0.010320,0.000153,0.078505,1.707301e-21,0.026567,0.036728,-2.272076e-09,0.000954,...,0.001273,0.005548,0.000009,0.048430,NaN,2.328306e-10,0.0,0.010379,0.000208,0.1250
2020-06-23,0.030900,0.018904,0.010442,0.000077,0.039253,8.536504e-22,0.030062,0.035031,4.901960e-03,0.002651,...,0.003913,0.002774,0.000005,0.024215,NaN,1.164153e-10,0.0,0.012972,0.318286,0.0625


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_deaths_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [81]:
jh_recovered_countries_hist_pct_ewma = jh_recovered_countries_hist_pct.ewm(span=3).mean()

In [82]:
jh_recovered_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-21,0.065373,0.010667,0.013585,0.000255,0.088595,0.001563,0.045419,0.046975,0.001230,0.001589,...,0.001396,0.000625,0.015779,0.002802,0.001261,0.008934,0.000651,0.141419,0.005981,0.010929
2020-06-22,0.037079,0.016356,0.014926,0.002653,0.044298,0.000781,0.039405,0.030653,0.001484,0.002153,...,0.001835,0.000927,0.016228,0.296012,0.002160,0.007884,0.000326,0.075032,0.002991,0.005465
2020-06-23,0.042236,0.023709,0.014181,0.001955,0.022149,0.000391,0.035783,0.070797,0.001393,0.001692,...,0.003943,0.000463,0.020474,0.148006,0.002604,0.003942,0.000163,0.046088,0.009452,0.002732


In [83]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_recovered_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [84]:
jh_cases_states_hist_pct_ewma = jh_cases_states_hist_pct.ewm(span=3).mean()

In [85]:
jh_cases_states_hist_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-21,0.019623,0.019521,NaN,0.058473,0.032257,0.021753,0.006568,0.001791,0.008024,2.148387e-30,...,0.019788,0.034937,0.029535,0.007212,0.020985,0.009932,0.017264,0.014308,0.012947,0.014884
2020-06-22,0.017023,0.013750,NaN,0.048327,0.032901,0.028292,0.005987,0.001190,0.006100,1.074194e-30,...,0.016318,0.038922,0.027481,0.005332,0.010493,0.009027,0.011944,0.010904,0.011490,0.021227
2020-06-23,0.019069,0.008194,NaN,0.058770,0.034948,0.032310,0.006056,0.001873,0.004298,5.370968e-31,...,0.018707,0.041288,0.024742,0.003096,0.005246,0.009037,0.014909,0.013485,0.010991,0.020369


In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_cases_states_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [87]:
jh_cases_counties_hist_pct_ewma = jh_cases_counties_hist_pct.ewm(span=3).mean()

In [88]:
jh_cases_counties_hist_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-21      0.024488  0.022844  0.011665  0.012872  0.042843  0.007203   
2020-06-22      0.021590  0.019856  0.015058  0.014501  0.035120  0.005145   
2020-06-23      0.023410  0.025331  0.012964  0.031060  0.037560  0.002572   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-06-21      0.007793  0.007460  0.013851  0.009997  ...  0.071185   
2020-06-22      0.005638  0.008608  0.013829  0.004998  ...  0.081047   
2020-06-23      0.005423  0.006719  0.012751  0.002499  ...  0.207190   

Province_State                                                         \
Admin2                Platte  Sheridan  Sublette Sweetwater     Teton   
2020-06-21     -2.646978e-23  0.000017  0.020833   0.061370  0.006988   
2020-06-22      5.000000e-01  0.000009  0.135417   0.058987  0.008039   
2020-06-23      2.500000e-01  0.000004  0.067708   0.047350  0.013029   

Province_State                                            
Admin2             Uinta Unassigned      Washakie Weston  
2020-06-21      0.062247        NaN  2.230762e-07    0.0  
2020-06-22      0.067355        NaN  1.115381e-07    0.0  
2020-06-23      0.047191        NaN  5.576906e-08    0.0  

[3 rows x 3254 columns]

In [89]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_cases_counties_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [90]:
jh_deaths_states_hist_pct_ewma = jh_deaths_states_hist_pct.ewm(span=3).mean()

In [91]:
jh_deaths_states_hist_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-21,0.009374,0.000101,NaN,0.012292,0.023265,0.008492,0.001709,0.002414,0.003305,NaN,...,0.010168,0.008747,0.014786,0.001136,1.477952e-13,0.003912,0.006019,0.005714,0.007939,0.013903
2020-06-22,0.005879,0.000050,NaN,0.006887,0.016077,0.008870,0.002069,0.001559,0.001653,NaN,...,0.009837,0.008025,0.007393,0.000568,7.389761e-14,0.004749,0.005372,0.002857,0.004642,0.006952
2020-06-23,0.016614,0.000025,NaN,0.020098,0.030065,0.009825,0.005274,0.002422,0.080137,NaN,...,0.015276,0.010582,0.019519,0.000284,3.694881e-14,0.010091,0.005821,0.018282,0.005676,0.003476


In [92]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_deaths_states_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [93]:
jh_deaths_counties_hist_pct_ewma = jh_deaths_counties_hist_pct.ewm(span=3).mean()

In [94]:
jh_deaths_counties_hist_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga       Baldwin Barbour Bibb Blount   Bullock   
2020-06-21      0.046180  5.858974e-11     0.0  0.0    0.0  0.015635   
2020-06-22      0.023090  2.929487e-11     0.0  0.0    0.0  0.007818   
2020-06-23      0.011545  1.464744e-11     0.0  0.0    0.0  0.003909   

Province_State                                          ... Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...    Park Platte   
2020-06-21      0.005028  0.005208  0.000687  0.025249  ...     NaN    NaN   
2020-06-22      0.002514  0.002604  0.000343  0.012625  ...     NaN    NaN   
2020-06-23      0.020488  0.126302  0.000172  0.089646  ...     NaN    NaN   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2020-06-21          NaN      NaN        NaN   NaN   NaN   0.014721      NaN   
2020-06-22          NaN      NaN        NaN   NaN   NaN   0.007361      NaN   
2020-06-23          NaN      NaN        NaN   NaN   NaN   0.003680      NaN   

Province_State         
Admin2         Weston  
2020-06-21        NaN  
2020-06-22        NaN  
2020-06-23        NaN  

[3 rows x 3254 columns]

In [95]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_deaths_counties_hist_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

# TODO - CONTINUE FROM HERE

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/states | [CSV](https://covidtracking.com/api/states.csv)  
States daily 4 pm ET - /api/states/daily | [CSV](http://covidtracking.com/api/states/daily.csv)  
States info - /api/states/info | [CSV](https://covidtracking.com/api/states/info.csv)  
US current - /api/us | [CSV](https://covidtracking.com/api/us.csv)  
US daily - /api/us/daily | [CSV](https://covidtracking.com/api/us/daily.csv)  
Counties- /api/counties | [CSV](https://covidtracking.com/api/counties.csv)  